### Урок 3. Условная и безусловная оптимизация#

### -- Автор: Шенк Евгений Станиславович

In [1]:
import numpy as np    
import math
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

### Задание 1.

Сталепрокатное предприятие выпускает стандартные стальные листы размерами 1500×6000 мм.  
Ежемесячный объем заказов на листы нестандартных размеров приведён ниже:  
500×600 — 3000, 1000×400 — 2000, 1500×1000 — 1500  
Необходимо составить ЦФ для нахождения наименее затратного раскроя листов.  

Целевая функция:  
$$\sum_{i=1}^{m}x_ia_i \rightarrow min$$

$x_i$ - количество труб, раскраиваемые i-м вариантом $(i=\overline{1,m})$  
$a_i$ - остаток трубы 

In [2]:
# Количество нестандартных листов:
n1 = 3000  
n2 = 2000  
n3 = 1500 

# Площадь листов (мм):
y0 = 9000000 # 1500x6000
y1 = 300000 # 500x600  
y2 = 400000 # 1000x400
y3 = 1500000 # 1500x1000

# Варианты раскроя: [обрезки, кол-во листов 500×600, 1000×400, 1500×1000]
a0 = [0, 30, 0, 0]
a1 = [0, 10, 15, 0]
a2 = [0, 0, 0, 6]
a3 = [0, 11, 3, 3] 
a4 = [0, 22, 6, 0]
a5 = [0, 15, 0, 3]

In [3]:
# Целевая функция (минимизируем кол-во обрезков) будет выглядеть:  
# a0[0]*x0 + a1[0]*x1 + a2[0]*x2 + a3[0]*x3 + a4[0]*x4 + a5[0]*x5 -> min
# ai[0] - обрезки от варианта i
# xi - количество использований варианта i

In [4]:
# Условия:
# a0[1]*x0 + a1[1]*x1 + a2[1]*x2 + a3[1]*x3 + a4[1]*x4 + a5[1]*x5 = 3000 
# a0[2]*x0 + a1[2]*x1 + a2[2]*x2 + a3[2]*x3 + a4[2]*x4 + a5[2]*x5 = 2000 
# a0[3]*x0 + a1[3]*x1 + a2[3]*x2 + a3[3]*x3 + a4[3]*x4 + a5[3]*x5 = 1500 

In [5]:
# Так как у нас все варианты имеют 0 обрезов, то минимальные затраты количества листов считаем как 
# Площадь готовых изделий деленая на площадь стандартного листа с округлением вверх

# Площадь готовых изделий:
S_product = n1 * y1 + n2 * y2 + n3 * y3

# Минимальное количество основных (1500х6000) листов покрывающее итоговую площадь готовых изделий:
math.ceil(S_product / y0)

439

Решение:  
т.к. у нас имеются имеются удобные варианты раскроя с 0 обрезков и позволяющие получить только нужный вид изделия  
1 - а0; 3 - а3; вид 2 имеется во варианте 2, то:  
1500//6 = 250 остаток 0 (получили все 1500 изделий вида 3)  
2000//15 = 133 остаток 5 (тоесть получили 1330 изделий 1 и 1995 изделий 2)  
(3000 - 1330) // 30 = 55 (получили все изделия вида 1)  
остаток: [0, 5, 0]  
остаток можно получить варинтом а1 или а4

In [6]:
x0 = 55
x1 = 133
x2 = 250
x3 = 0
x4 = 1
x5 = 0

x0 + x1 + x2 + x3 + x4 + x5

439

Полученный результат соответствует минимальным затратам

### Задание 2.

Пусть есть компания, занимающаяся доставкой Belivery Club.  
Есть N зон, в которых есть спрос на курьеров в каждый час. Внутри дня в разные часы спрос различен. Каждой зоне i в час h необходимо необходимо $n_{hi}$ курьеров. Для каждой зоны есть штраф из-за недостаточно 1 курьера - $s_i$. Так же у каждого курьера есть предпочтения относительно часов и зон работы. Необходимо распределить M курьеров таким образом, чтобы суммарный штраф был минимальный.
(описать формулу в общем случае, т.е. без каких-либо числовых данных)

Необходимо распределить M курьеров таким образом, чтобы суммарный штраф был минимальный.



Пронумеруем курьеров:
пусть $x_{ih}^k$ = 1, если k-курьер работает в час h в i зоне

Тогда в одной зоне в час h работает $$\sum_{k=1}^{M} x_{ih}^k$$

Введем штрафы за попадание курьеров в непредпочитаемые часы $h_e$ штраф - $t_i$ и зоны $i_e$ штраф - $r_i$ (доплата за закрытие нам недостающих курьеров в другие часы и других зонах)

Тогда штраф $s_e$ за непредпочитаемые часы и зоны:  
$$s_e = \sum_{k=1}^{M} x_{ih_e}^k * t_{i} + \sum_{k=1}^{M} x_{i_e h}^k * r_{i}$$


Штраф в час в этой зоне
$$ f_{ih} = \begin{cases} s_e, \space если \space \sum_{k=1}^{M} x_{ih}^k - n_{ih} >0 \\
s_e + s_i * (\sum_{k=1}^{M} x_{ih}^k - n_{ih}), в\spaceпротивном\spaceслучае. \end {cases}$$ 


Штраф за день $$\sum_{h=1}^{24}f_{ih}$$

Штраф по всем зонам в течении дня:


$$\sum_{i=1}^{N}\sum_{h=1}^{24}f_{ih}$$

Целевая функция:

$$\sum_{i=1}^{N}\sum_{h=1}^{24}f_{ih} \rightarrow min $$